# Lennard-Jones Liquid Analysis and Visualization

This notebook provides tools for analyzing and visualizing the results of LJ liquid simulations.

In [ ]:
# Install required packages
import subprocess
import sys

packages = ['matplotlib', 'numpy', 'pandas', 'scipy']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("All dependencies installed successfully!")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import os

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## Load Simulation Data

In [ ]:
# Load thermodynamic data
output_dir = '../output'

# Check if files exist
files_exist = {
    'energy': os.path.exists(f'{output_dir}/energy.csv'),
    'pressure': os.path.exists(f'{output_dir}/pressure.csv'),
    'temperature': os.path.exists(f'{output_dir}/temperature.csv')
}

print("Available data files:")
for name, exists in files_exist.items():
    print(f"  {name}.csv: {'✓' if exists else '✗'}")

# Load data
if files_exist['energy']:
    energy_data = pd.read_csv(f'{output_dir}/energy.csv')
    print(f"\nEnergy data: {len(energy_data)} rows")
    print(energy_data.head())

if files_exist['pressure']:
    pressure_data = pd.read_csv(f'{output_dir}/pressure.csv')
    print(f"\nPressure data: {len(pressure_data)} rows")
    print(pressure_data.head())

if files_exist['temperature']:
    temp_data = pd.read_csv(f'{output_dir}/temperature.csv')
    print(f"\nTemperature data: {len(temp_data)} rows")
    print(temp_data.head())

## Thermodynamic Property Evolution

In [ ]:
# Create comprehensive thermodynamic plots
n_plots = sum([files_exist['temperature'], files_exist['energy'], files_exist['pressure']])

if n_plots == 0:
    print("No data files found for plotting.")
else:
    fig = plt.figure(figsize=(14, 4*n_plots))
    gs = GridSpec(n_plots, 1, figure=fig, hspace=0.3)
    
    plot_idx = 0
    
    # Temperature plot
    if files_exist['temperature']:
        ax = fig.add_subplot(gs[plot_idx, 0])
        ax.plot(temp_data['Step'], temp_data['Temperature'], linewidth=1.5, color='#e74c3c')
        ax.set_xlabel('Timestep')
        ax.set_ylabel('Temperature (T*)')
        ax.set_title('Temperature Evolution', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        # Add mean temperature line
        mean_temp = temp_data['Temperature'].mean()
        ax.axhline(mean_temp, color='black', linestyle='--', linewidth=1, 
                   label=f'Mean: {mean_temp:.4f}')
        ax.legend()
        plot_idx += 1
    
    # Energy plot
    if files_exist['energy']:
        ax = fig.add_subplot(gs[plot_idx, 0])
        ax.plot(energy_data['Step'], energy_data['PE'], label='Potential Energy', 
                linewidth=1.5, color='#3498db')
        ax.plot(energy_data['Step'], energy_data['KE'], label='Kinetic Energy', 
                linewidth=1.5, color='#2ecc71')
        ax.plot(energy_data['Step'], energy_data['TotalEnergy'], label='Total Energy', 
                linewidth=1.5, color='#9b59b6')
        ax.set_xlabel('Timestep')
        ax.set_ylabel('Energy (ε)')
        ax.set_title('Energy Evolution', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plot_idx += 1
    
    # Pressure plot
    if files_exist['pressure']:
        ax = fig.add_subplot(gs[plot_idx, 0])
        ax.plot(pressure_data['Step'], pressure_data['Pressure'], 
                linewidth=1.5, color='#f39c12')
        ax.set_xlabel('Timestep')
        ax.set_ylabel('Pressure (P*)')
        ax.set_title('Pressure Evolution', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        # Add mean pressure line
        mean_press = pressure_data['Pressure'].mean()
        ax.axhline(mean_press, color='black', linestyle='--', linewidth=1, 
                   label=f'Mean: {mean_press:.4f}')
        ax.legend()
        plot_idx += 1
    
    plt.savefig(f'{output_dir}/thermodynamics.png', dpi=300, bbox_inches='tight')
    plt.show()
    print(f"\nPlot saved to {output_dir}/thermodynamics.png")

## Statistical Analysis

In [ ]:
# Calculate and display statistics
print("="*60)
print("SIMULATION STATISTICS")
print("="*60)

if files_exist['temperature']:
    print("\nTemperature:")
    print(f"  Mean: {temp_data['Temperature'].mean():.6f}")
    print(f"  Std Dev: {temp_data['Temperature'].std():.6f}")
    print(f"  Min: {temp_data['Temperature'].min():.6f}")
    print(f"  Max: {temp_data['Temperature'].max():.6f}")

if files_exist['energy']:
    print("\nEnergy:")
    print(f"  Mean PE: {energy_data['PE'].mean():.6f}")
    print(f"  Mean KE: {energy_data['KE'].mean():.6f}")
    print(f"  Mean Total: {energy_data['TotalEnergy'].mean():.6f}")
    print(f"  Energy Drift: {abs(energy_data['TotalEnergy'].iloc[-1] - energy_data['TotalEnergy'].iloc[0]):.6e}")

if files_exist['pressure']:
    print("\nPressure:")
    print(f"  Mean: {pressure_data['Pressure'].mean():.6f}")
    print(f"  Std Dev: {pressure_data['Pressure'].std():.6f}")
    print(f"  Min: {pressure_data['Pressure'].min():.6f}")
    print(f"  Max: {pressure_data['Pressure'].max():.6f}")

print("\n" + "="*60)

## Radial Distribution Function (RDF)

Calculate the pair correlation function g(r) from the trajectory.

In [ ]:
# Load and plot RDF data
rdf_file = f'{output_dir}/rdf.dat'

if os.path.exists(rdf_file):
    # Read RDF data (LAMMPS format: # Row r g(r))
    rdf_data = []
    with open(rdf_file, 'r') as f:
        for line in f:
            if not line.startswith('#') and line.strip():
                parts = line.split()
                if len(parts) >= 3:
                    row, r, gr = parts[0], float(parts[1]), float(parts[2])
                    rdf_data.append([r, gr])
    
    rdf_array = np.array(rdf_data)
    r_values = rdf_array[:, 0]
    g_values = rdf_array[:, 1]
    
    # Plot RDF
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(r_values, g_values, linewidth=2, color='#2c3e50')
    ax.set_xlabel('r/σ', fontsize=12)
    ax.set_ylabel('g(r)', fontsize=12)
    ax.set_title('Radial Distribution Function', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.axhline(1.0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='g(r) = 1')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/rdf.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"RDF plot saved to {output_dir}/rdf.png")
    print(f"\nRDF Statistics:")
    print(f"  Number of bins: {len(r_values)}")
    print(f"  Maximum g(r): {g_values.max():.4f} at r/σ = {r_values[np.argmax(g_values)]:.4f}")
    print(f"  First minimum at r/σ ≈ {r_values[np.argmin(g_values[:50])]:.4f}")
    
    # Find coordination number (integral of 4πr²ρg(r) up to first minimum)
    density_param = 0.8  # This should match input density
    first_min_idx = np.argmin(g_values[:50])
    r_first_min = r_values[first_min_idx]
    
    # Numerical integration using trapezoidal rule
    r_integr = r_values[:first_min_idx+1]
    g_integr = g_values[:first_min_idx+1]
    integrand = 4 * np.pi * r_integr**2 * density_param * g_integr
    coord_num = np.trapz(integrand, r_integr)
    
    print(f"  Coordination number (up to first minimum): {coord_num:.2f}")
else:
    print("RDF data file not found. Make sure the simulation completed successfully.")
    print(f"Expected file: {rdf_file}")